This report provides status information on the most recent endpoints for a hardcoded list of prioritised list of LPAs, or organisations from an input.

The input should be called 'input.csv' and contain one column, 'organisation' that has the organisation codes for the LPAs to be included in the report.

In [11]:
import urllib
import pandas as pd
import urllib.parse
import os
%pip install wget
import wget

Note: you may need to restart the kernel to use updated packages.


In [12]:
util_file = "master_report_endpoint_utils.py"
if os.path.isfile(util_file):
    from master_report_endpoint_utils import *
else:
    url = "https://raw.githubusercontent.com/digital-land/jupyter-analysis/main/service_report/master_report/master_report_endpoint_utils.py"
    wget.download(url)
    from master_report_endpoint_utils import *

In [13]:
# Get input from .csv or use default prioritised LPAs
input_path = './input.csv'
if os.path.isfile(input_path):
    input_df = pd.read_csv(input_path)
    organisation_list = input_df['organisation'].tolist()
    print('Input file found. Using', len(organisation_list), 'organisations from input file.')
else:
    organisation_list = ['local-authority-eng:BUC', 'local-authority-eng:DAC', 'local-authority-eng:DNC',
    'local-authority-eng:GLO', 'local-authority-eng:CMD', 'local-authority-eng:LBH', 'local-authority-eng:SWK',
    'local-authority-eng:MDW', 'local-authority-eng:NET', 'local-authority-eng:BIR', 'local-authority-eng:CAT',
    'local-authority-eng:EPS', 'local-authority-eng:BNE', 'local-authority-eng:GAT', 'local-authority-eng:GRY',
    'local-authority-eng:KTT', 'local-authority-eng:SAL', 'local-authority-eng:TEW', 'local-authority-eng:WBK',
    'local-authority-eng:DST', 'local-authority-eng:DOV', 'local-authority-eng:LIV', 'local-authority-eng:RDB',
    'local-authority-eng:WFT', 'local-authority-eng:NLN', 'local-authority-eng:NSM', 'local-authority-eng:SLF',
    'local-authority-eng:WRL' ]
    print('Input file not found. Using default list of organisations.')

Input file not found. Using default list of organisations.


In [14]:
# Get organisation info
organisation_info_df = pd.read_csv('https://raw.githubusercontent.com/digital-land/organisation-collection/main/data/local-authority.csv')
organisation_info_df.head()
organisation_name_dict = {}
for organisation in organisation_list:
    organisation_code = organisation.split(':')[1]
    organisation_name = organisation_info_df.loc[organisation_info_df['reference'] == organisation_code].iloc[0]['name']
    organisation_name_dict[organisation] = organisation_name

In [15]:
datasette_url = "https://datasette.planning.data.gov.uk/"

def update_dataframe(organisation):
    global result_df  
    if organisation:
        query = f" s.organisation = '{organisation}'"
    else:
        query = f" s.organisation LIKE '%'"
    params = urllib.parse.urlencode({
        "sql": f"""
        select
          e.endpoint_url,
          l.status,
          l.exception,
          s.collection,
          group_concat(DISTINCT sp.pipeline) as pipelines,
          s.organisation,
          o.name,
          max(l.entry_date) maxentrydate,
          max(e.entry_date) entrydate,
          e.end_date
        from
          log l
          inner join source s on l.endpoint = s.endpoint
          inner join organisation o on s.organisation=o.organisation
          inner join endpoint e on l.endpoint = e.endpoint
          inner join source_pipeline sp on s.source = sp.source
        where
           {query} and not collection="brownfield-land"
        group by
          l.endpoint,
          l.status
        order by
          l.endpoint,
          s.collection,
          maxentrydate desc
        """,
        "_size": "max"
    })
    
    url = f"{datasette_url}digital-land.csv?{params}"
    df = pd.read_csv(url)
    result_df = df
    return df

def update_dataframe_latest_status(organisation):
    global new_df
    all_endpoints=update_dataframe(organisation)
    new_df=all_endpoints.copy()
    new_df['maxentrydate'] = pd.to_datetime(new_df['maxentrydate'])
    new_df['last_status'] = None
    new_df['last_updated_date'] = None
    new_df['date_last_status_200'] = None
    
    for index, row in new_df.iterrows():
        if index < len(new_df) - 1 and (row['status']!=200 or pd.isna(row['status'])):
            if row['endpoint_url'] == new_df.at[index + 1, 'endpoint_url']:
                new_df.at[index, 'last_status'] = new_df.at[index + 1, 'status']
                new_df.at[index, 'last_updated_date'] = new_df.at[index + 1, 'maxentrydate']   
    
    new_df.drop_duplicates(subset='endpoint_url', keep='first', inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    for index, row in new_df.iterrows():
        if row['last_status'] is not None:
            if row['last_status'] != 200  or row['last_status'] is None:
                filtered_df = all_endpoints[(all_endpoints['endpoint_url'] == row['endpoint_url'] ) & (all_endpoints['status'] == 200)]
                if not filtered_df.empty:
                    new_df.at[index, 'date_last_status_200'] = filtered_df['maxentrydate'].values[0][:19] 
    return new_df

collection_list = ['article-4-direction', 'article-4-direction-area', 'conservation-area', 'conservation-area-document', 'listed-building-outline', 'tree-preservation-order', 'tree-preservation-zone', 'tree']
pipelines_list = ['article-4-direction', 'article-4-direction-area', 'conservation-area', 'conservation-area-document', 'listed-building-outline', 'tree-preservation-order', 'tree-preservation-zone', 'tree', 'tree,tree-preservation-order', 'tree-preservation-order,tree-preservation-zone','article-4-direction,article-4-direction-area']
all_orgs_recent_endpoints={}
for organisation in organisation_list:
    recent_endpoints_df = update_dataframe_latest_status(organisation)
    recent_endpoints_df = recent_endpoints_df[recent_endpoints_df['pipelines'].isin(pipelines_list)]
    all_orgs_recent_endpoints[organisation] = recent_endpoints_df


In [16]:
update_dataframe("local-authority-eng:BUC").head()

,endpoint_url,status,exception,collection,pipelines,organisation,name,maxentrydate,entrydate,end_date
0,https://maps.buckscc.gov.uk/arcgis/services/PL...,NaN,ConnectTimeout,listed-building-outline,listed-building-outline,local-authority-eng:BUC,Buckinghamshire Council,2023-11-17T00:16:29Z,2022-05-04T16:16:06Z,NaN
1,https://maps.buckscc.gov.uk/arcgis/services/PL...,200.0,NaN,listed-building-outline,listed-building-outline,local-authority-eng:BUC,Buckinghamshire Council,2023-11-16T00:16:27Z,2022-05-04T16:16:06Z,NaN
2,https://raw.githubusercontent.com/digital-land...,200.0,NaN,design-code,design-code-area,local-authority-eng:BUC,Buckinghamshire Council,2023-11-17T00:11:54Z,2022-05-10T18:18:25Z,NaN
3,https://maps.buckscc.gov.uk/arcgis/services/PL...,200.0,NaN,article-4-direction-area,article-4-direction-area,local-authority-eng:BUC,Buckinghamshire Council,2023-11-17T00:15:42Z,2022-04-28T16:16:57Z,NaN
4,https://maps.buckscc.gov.uk/arcgis/services/PL...,NaN,ConnectTimeout,article-4-direction-area,article-4-direction-area,local-authority-eng:BUC,Buckinghamshire Council,2023-11-15T00:15:38Z,2022-04-28T16:16:57Z,NaN


In [17]:
pd.set_option('display.max_colwidth', None)

def compute_cell_colour(status):
    if status == 200:
        return 'background-color: green'
    elif status == 'No endpoint':
        return 'background-color: orange'
    else:
        return 'background-color: red'
    
def compute_status(existing_statuses, collection, status):
    if collection in existing_statuses:
        if existing_statuses[collection] == 200:
            return status
        elif existing_statuses[collection] == 'No endpoint':
            return status
        else:
            return existing_statuses[collection]
    else:
        return status

rows_list = []
organisation_dataset_statuses_dict = {}
for organisation in organisation_list:
    # organisation = "local-authority-eng:BUC"
    df = all_orgs_recent_endpoints[organisation]
    df = df[pd.isna(df['end_date'])]
    try:
        name = organisation_name_dict[organisation]
    except:
        name = organisation
    dataset_statuses_dict = {}
    for pipeline in pipelines_list:
        status = df[df['pipelines'] == pipeline]['status']
        df_datasets = pd.DataFrame(df['pipelines'], columns=["pipelines"])
        list_datasets = df_datasets['pipelines'].tolist()
        if ',' in pipeline:
            if pipeline in list_datasets:
                datasets = pipeline.split(',')
                status = pd.Series(status)
                if status.empty or (status=='No endpoint').any():
                    status = 'No endpoint'
                else:
                    if not pd.isna(status.values[0]):
                        status = int(status.values[0])
                    else:
                        status=df.loc[df['status'].isna(), 'exception'].values[0]
                        if status is None:
                            status="Unknown Error"
                for i in datasets:
                    dataset_statuses_dict[i] = compute_status(dataset_statuses_dict, i, status)
        else:
            if status.empty:
                status = 'No endpoint'
            else:
                df_status = status.to_frame()
                df_no_duplicates = df_status.drop_duplicates()
                df_filtered = df_no_duplicates[df_no_duplicates['status'] != 200]
                if df_filtered.size>0:
                    if not pd.isna(df_filtered.iloc[0, 0]):
                        status=int(df_filtered.iloc[0, 0])
                    else:
                        status=df.loc[df['status'].isna(), 'exception'].values[0]
                        if status is None:
                            status="Unknown Error"
                else:
                    status=200
            dataset_statuses_dict[pipeline] = compute_status(dataset_statuses_dict, pipeline, status)
    organisation_dataset_statuses_dict[organisation] = dataset_statuses_dict
    new_row = {'organisation': name}
    new_row.update(dataset_statuses_dict)
    rows_list.append(new_row)

output_df = pd.DataFrame(rows_list, columns=['organisation', *collection_list])
output_df.to_csv('endpoint_status_master_report.csv', index=False)
output_df.style.applymap(compute_cell_colour, subset=collection_list)

NameError: name 'statuses' is not defined

In [ ]:
output_columns = ['name', 'pipelines', 'endpoint_url', 'organisation', 'collection', 'maxentrydate', 'entrydate', 'end_date', 'last_status', 'last_updated_date']

output_df = produce_output_csv(all_orgs_recent_endpoints, orga, "issues", "No issues", output_columns)
